# About Needs

On Infinity, the Needs represent the conditions for goals. However, being that we are formulating the system for society define its needs at scale, we have to treat society's institutions as the systems that are efficient at satisfying needs. Thus, if we apply the `yml` syntax, commonly software infrastructure defining, the infinity script that could be included within a Goal (`Topic.type=1`), could look like this:


```inf
- target: agents:construction_companies_N101
  needs:
  - name: ensure the companies have the level P110 permissions to build in the open sea
    united_nations: permission=P110 state=present
- target: places:construction_companies_N101
  - name: ensure the companies have high grade concrete type X123
    scoutbee_gmbh: cement=X123 state=present
```

Note, that, just like at the level of Ansible, we use package managers as our tools to satisfy conditions, here we use the actual institutions or companies as tools that are adept at the specific areas to require to satisfy them.


In [157]:
import mistune
import bs4
import yaml

targets = bs4.BeautifulSoup(
    mistune.markdown(
        '''
```inf
- target: agents:construction_companies_N101
  needs:
  - name: ensure the companies have the level P110 permissions to build in the open sea
    united_nations: permission=P110 state=present
  - name: something new
    world_trade_organization: some=XXX state=present
- target: places:construction_companies_N101
  needs:
  - name: ensure the companies have high grade concrete type X123
    scoutbee_gmbh: cement=X123 state=present
- target: agents:humans
  needs:
  - name: ensure universal basic income to all
    world_economic_sustainability_organization: ubi=x001 state=present
```
        '''
    ),
    'html.parser'
).find_all('code', {'class': 'lang-inf'})

conditions = []

get_slice = lambda _, excluding: {
    key: _[key] for key in [key for key in _.keys() if key not in excluding]}

for target in targets: 
    for block in yaml.load(target.text):
        if 'needs' in block.keys():
            items = block.get('needs')
            if 'target' in block.keys():
                for i, _ in enumerate(items):
                    items[i].update(
                        {'target': block['target']}
                    )
                    tools = get_slice(items[i], excluding=['name', 'target'])
                    items[i].update(
                        {'tool': tools}
                    )
                    for key, tool in enumerate(tools):
                        del items[i][tool]
            conditions.extend( items )
conditions

[{'name': 'ensure the companies have the level P110 permissions to build in the open sea',
  'target': 'agents:construction_companies_N101',
  'tool': {'united_nations': 'permission=P110 state=present'}},
 {'name': 'something new',
  'target': 'agents:construction_companies_N101',
  'tool': {'world_trade_organization': 'some=XXX state=present'}},
 {'name': 'ensure the companies have high grade concrete type X123',
  'target': 'places:construction_companies_N101',
  'tool': {'scoutbee_gmbh': 'cement=X123 state=present'}},
 {'name': 'ensure universal basic income to all',
  'target': 'agents:humans',
  'tool': {'world_economic_sustainability_organization': 'ubi=x001 state=present'}}]

This produces the list of needs, that are to be linked, or updated based on uniqueness of  `title=name`, by creating needs, like so:

In [158]:
len(conditions)

4

In [80]:
current_topic

<Topic: [Goal] .:en:Reduce risk regarding subjective head internal pressure>

In [86]:
Topic.objects.create(
    title='abc',
    body='hello',
    owner=current_topic.owner
)

<Topic: [Task] .:so:abc>

In [95]:
t = Topic.objects.get(title='.:so:abc')

In [97]:
current_topic.parents.add(t)

In [100]:
t.title == '.:so:abc'

True

In [107]:
current_topic.parents.filter(title='.:so:abc').exists()

True

In [83]:
title='.:en:{}'.format(conditions[0]['name'])

In [84]:
title

'.:en:ensure the companies have the level P110 permissions to build in the open sea'

In [155]:
from src.core.models import Topic

current_topic = Topic.objects.get(title='.:en:Reduce risk regarding subjective head internal pressure')
lang = 'en'

for condition in conditions:
    
    title='.:{}:{}'.format(lang, condition['name'])
    body = '.:{}\n{}'.format(lang, yaml.dump(condition, default_flow_style=False))
    
    if not current_topic.parent_topics.filter(title=title).exists():
        topic = Topic.objects.create(
            title=title,
            body=body,
            owner=current_topic.owner,
            type=0
        )
        current_topic.parent_topics.add(topic)
        current_topic.save()
    else:
        
        topic = current_topic.parent_topics.filter(title=title).first()
        topic.body = body
        topic.save()

for need in current_topic.parent_topics.filter(type=0):
    if need.title not in ['.:{}:{}'.format(lang, d['name']) for d in conditions]:
        need.delete()

In [156]:
for parent in current_topic.parent_topics.all():
    print(parent)

[Need] .:en:something new
[Need] .:en:ensure universal basic income to all
[Need] .:en:ensure the companies have high grade concrete type X123
[Need] .:en:ensure the companies have the level P110 permissions to build in the open sea
[Idea] .:en:Maybe do MRI scan to see if there are any problems with blood passage
[Idea] .:en:Maybe do intracranial pressure measurement


In [159]:
current_topic.parent_topics.count()

6